# Deskripsi
Deskripsi mengenai variable-variabel yang ada dalam penelitian ini akan dijelaskan dalam beberapa bagian:

1. Labeling
2. Loading
3. 'News' Variables Description

### Karakteristik Ekonomi 
##### (Eurostat Business Register,2010)
1. Klasifikasi Aktivitas Ekonomi (KBLI)
    * Aktivitas ekonomi berdasarkan bahan produksi
2. Nilai tambah Perusahaan
3. Ukuran-ukuran Perusahaan
    * Jumlah pekerja
    * *Turnover* (Nilai ambil alih)
4. Ukuran-ukuran Perusahaan berdasarkan Balance sheet
    * Aset
    * Inventori
    * Cash
    * Dividen
    * Obligasi
5. Ukuran Perusahaan berdasarkan Upah dan Gaji
    * Upah (*Wage*)
    * Gaji (*Salary*)

### Karakteristik Demografi
##### (Eurostat Business Register,2010)
1. Active and Restart
    * Birth
    * Death
    * Reactivation
    * Real Death and Real Birth
2. Survive
    * Break-Up
    * Split-Off
    * Merger
    * Takeover
3. Structure
    * Change of Ownership
    * Change of Group
    * Restructure within enterprise/enterprise group
    * Complex Restructure

### Karakteristik Link dan Acuan Eksternal
##### (Pedoman Statistical Business Registers,2015)
1. Persentase Kepemilikan
2. Hubungan antar 2 perusahaan
    * Anak perusahaan

In [2]:
import matplotlib.pyplot as plt
import os, re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from datetime import datetime as dt
import cPickle as pickle
import re #Regex
import seaborn as sns
import pandas as pd
sns.set()
%matplotlib inline

In [3]:
DATAStem = pd.DataFrame()
DATAPost = pd.DataFrame()

#Load from PosTagged Text
%time DATAPost['Judul'] = pickle.load( open( "v1.4\DATA_JUDULStriped-v2.p", "rb" ) )
%time DATAPost['Short'] = pickle.load( open( "v1.4\DATA_SHORTStriped-v2.p", "rb" ) )
%time DATAPost['Long']  = pickle.load( open( "v1.4\DATA_LONGStriped-v2.p", "rb" ) )

%time DATA_TARGET = pickle.load( open( "v1.4\DATA_TARGET-v2.p", "rb" ) )

DATA_TARGET = DATA_TARGET.sort_index()
DATAPost = DATAPost.sort_index()

DATA_TARGET = DATA_TARGET.reset_index(drop=True)
DATAPost = DATAPost.reset_index(drop=True)

Wall time: 445 ms
Wall time: 562 ms
Wall time: 4.38 s
Wall time: 139 ms


In [4]:
%time postagger = pickle.load(open( "POSTAGGER.p", "rb" ))
%time tagList   = pickle.load(open( "TAGLIST.p", "rb" ))

import string
import operator

def toNVFraction(data):
    def combine_dicts(a, b, op=operator.add):
        return dict(a.items() + b.items() +
            [(k, op(a[k], b[k])) for k in set(b) & set(a)])
    
    def onlyAZ(teks):
        return re.sub(r'[^a-zA-Z]', ' ', teks)
    
    def onlyNVFractionSentence(teks):
        splitted = postagger.tag(onlyAZ(teks).split())
        N  = len(splitted)
        word = 0
        k = {}
        for i in tagList:k[i] = float (0)
        for word,pos in splitted:
            k[pos]+=1
        return N,k

    def onlyNVFractionParagraph(par):
        splittedPar = par.split('.')
        word = 0
        _k = {}
        for i in tagList:_k[i] = float (0)
        for sentence in splittedPar:
            w,k = onlyNVFractionSentence(sentence)
            word+=w
            _k = {x: _k.get(x, 0) + k.get(x, 0) for x in set(_k)}
        #return noun/word,verb/word
        return _k.values()

    
    balikan = np.array([0 for i in range(0,len(tagList))])
    for teks in data:
        frac = onlyNVFractionParagraph(teks)
        balikan = np.vstack((balikan,frac))
    return balikan[1:]

Wall time: 362 ms
Wall time: 0 ns


In [5]:
%time JUDULFrac = toNVFraction(DATAPost['Judul'])
%time SHORTFrac = toNVFraction(DATAPost['Short'])
%time LONGFrac  = toNVFraction(DATAPost['Long'])

Wall time: 58.5 s
Wall time: 1min 1s
Wall time: 2min 15s


In [6]:
cek = DATAPost.Short[DATA_TARGET[DATA_TARGET == 1].index].head()
cek

13     perseroan mengaku 2015 tahun yang sulit karena...
130        tetap dikawal jangan sampai merugikan negara.
197                        laba anjlok sampai 75 persen.
210    jakarta   bisnis bioskop ternyata memberikan k...
940    pt astra sedaya finance tbk (asdf) berencana m...
Name: Short, dtype: object

In [7]:
for i in cek:
    print postagger.tag(re.sub(r'[^a-zA-Z]', ' ', i).split())

[('perseroan', 'NN'), ('mengaku', 'VB'), ('tahun', 'NN'), ('yang', 'SC'), ('sulit', 'RB'), ('karena', 'CC'), ('harga', 'NN'), ('batu', 'NN'), ('bara', 'NN'), ('anjlok', 'NN')]
[('tetap', 'JJ'), ('dikawal', 'NN'), ('jangan', 'NEG'), ('sampai', 'VB'), ('merugikan', 'VB'), ('negara', 'NN')]
[('laba', 'NN'), ('anjlok', 'NN'), ('sampai', 'VB'), ('persen', 'JJ')]
[('jakarta', 'NN'), ('bisnis', 'NN'), ('bioskop', 'NN'), ('ternyata', 'RB'), ('memberikan', 'VB'), ('kontribusi', 'CD'), ('signifikan', 'NN'), ('bagi', 'IN'), ('grup', 'NN'), ('lippo', 'NN'), ('tahun', 'NN'), ('lalu', 'JJ'), ('bisnis', 'NN'), ('itu', 'DT'), ('berkontribusi', 'CD'), ('sebanyak', 'NN'), ('rp', 'NN'), ('miliar', 'NN'), ('untuk', 'IN'), ('induk', 'NN'), ('usahanya', 'NN'), ('yakni', 'NN'), ('pt', 'NN'), ('first', 'NN'), ('media', 'VB'), ('tbk', 'NN'), ('klbv', 'NN')]
[('pt', 'NN'), ('astra', 'NN'), ('sedaya', 'NN'), ('finance', 'NN'), ('tbk', 'NN'), ('asdf', 'NN'), ('berencana', 'VB'), ('melakukan', 'VB'), ('pembayaran'

In [8]:
SHORTFrac[DATA_TARGET[DATA_TARGET == 1].index][:5]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   6.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   0.,   0.,   0.,   1.,   0.,   1.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   1.],
       [  0.,   1.,   1.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   1.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   1.,   0.,   1.,   0.,   0.,  18.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   0.,   0.,   0.,   2.,   0.,   0.,   2.,
          2.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  11.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [9]:
pickle.dump( JUDULFrac, open( "v2.0\DATA_JUDUL-v3.p", "wb" ) )
pickle.dump( SHORTFrac, open( "v2.0\DATA_SHORT-v3.p", "wb" ) )
pickle.dump( LONGFrac,  open( "v2.0\DATA_LONG-v3.p", "wb" ) )